In [123]:
import os, glob
import json, pickle
import nltk
import re
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer

from collections import Counter
import tqdm

In [2]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('tagsets')
# nltk.download('stopword')
# nltk.download('maxent_treebank_pos_tagger')
# nltk.download('averaged_perceptron_tagger')

In [3]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-NLP'

In [4]:
os.chdir('./../airbnb-data')

In [6]:
file_lst = glob.glob('translated_*.json')
file_lst

['translated_busan.json',
 'translated_daegu.json',
 'translated_daejeon.json',
 'translated_gangneung.json',
 'translated_gwangju.json',
 'translated_jeju.json',
 'translated_jeonju.json',
 'translated_seogwipo.json',
 'translated_seoul.json',
 'translated_seoul1.json',
 'translated_seoul2.json',
 'translated_seoul3.json',
 'translated_yeosu.json']

In [7]:
review = {}
for file in file_lst:
    with open(file,'r',encoding='utf-8') as fp:
        review.update(json.load(fp))

In [8]:
len(review)

32957

In [53]:
review['10030136'][0]

{'collection_tag': None,
 'comments': '친구들이랑 6명이 갔는데\n숙소 위치도 너무 좋고 방도 깨끗해서\n다들 맘에 들어했어요!!\n에어비엔비를  처음 이용해봤는데\n너무 좋아서 다음번에도 이용할것같아요\n해운대해변이랑도 걸어갈수 있고\n연락도 너무 잘됐어요',
 'created_at': '2016-10-08T23:57:42Z',
 'id': 106917581,
 'language': 'ko_en',
 'localized_date': '2016년 10월',
 'rating': 5,
 'response': '',
 'reviewee': {'deleted': False,
  'first_name': 'Jaewook',
  'host_name': 'Jaewook',
  'id': 38530724,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/users/38530724/profile_pic/1436955527/original.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/38530724'},
 'reviewer': {'deleted': False,
  'first_name': 'Joohee',
  'host_name': 'Joohee',
  'id': 96535213,
  'is_superhost': False,
  'picture_url': 'https://a0.muscache.com/im/pictures/32cbca58-85f2-47e5-b09c-a5ead7662304.jpg?aki_policy=profile_x_medium',
  'profile_path': '/users/show/96535213'},
 't_comments': 'I went to the dormitory with my friends. They liked the room because it was so nice and

In [54]:
review_en = {}
for home_id,rev_lst in review.items():
    review_en[home_id] = []
    for rev in rev_lst:
        if  rev['language'] is not None and rev['language'][-3:]=='_en':
            review_en[home_id].append([rev['rating'],rev['t_comments']])
        else:
            continue

In [55]:
review_en['10030136']

[[5,
  'I went to the dormitory with my friends. They liked the room because it was so nice and clean!This is my first time using Air Bienby. I love it, so I think I will use it next time.I can walk to Haeundae beach and the contact was great.'],
 [5,
  "The room was spacious and spacious, and the two bedrooms could be seen from the window.When we went there, we caught the typhoon and fortunately the room was occupied with double-decked windows.The room is modern and technological.Overall, it's pretty good."],
 [5,
  "When I arrived, my family praised me for my clean and spacious accommodation! The front was a construction site, so the view wasn't 100 percent cool, but it was enough to last.Ha"],
 [5,
  "The room is prettier and more sunny than the one you see in the picture. Especially, Haeundae Beach looks so peaceful, with two harmonics in the living room brightening the atmosphere of the interior.Thanks to the host's consideration, I could take a comfortable trip to Busan.Thank you

In [ ]:
## spacing 필요하지않을까??

In [56]:
re.sub('[^a-z|A-Z|0-9|.!?,|\s]','','asdlk. fj!          asdfㅁㄴ 아asd  23ㅁㄴㅇ!!@as')

'asdlk. fj!          asdf asd  23!!as'

In [57]:
for k,rev_lst in review_en.items():
    for i,rev in enumerate(rev_lst):
        rev_lst[i][1] = re.sub('[^a-z|A-Z|0-9|.!?,|\s]','',rev[1].strip())

In [63]:
# breakpoint = 0
# for k,rev_lst in review_en.items():
#     for rev in rev_lst:
#         if breakpoint == 1000:
#             break
#         print(rev[1],end='\n\n')
#         breakpoint+=1

In [62]:
sent_tokenize(review_en['10030136'][1][1]) # sent_tokenize 안되는 리뷰 꽤 있음

['The room was spacious and spacious, and the two bedrooms could be seen from the window.When we went there, we caught the typhoon and fortunately the room was occupied with doubledecked windows.The room is modern and technological.Overall, its pretty good.']

In [64]:
%%time
# 리뷰별 문장단위 tokenizing
for home_id,rev_lst in review_en.items():
    for rev in rev_lst:
        rev[1] = sent_tokenize(rev[1])

Wall time: 31 s


In [65]:
%%time 
# 문장별 단어단위 tokenizing
for home_id,rev_lst in review_en.items(): 
    for rev in rev_lst:
        tmp_lst = []
        for sent in rev[1]:
            word_lst = nltk.word_tokenize(sent)
            tmp_lst.append(word_lst)
        rev[1] = tmp_lst

Wall time: 2min 13s


In [66]:
nltk.pos_tag(review_en['10030136'][1][1][0])

[('The', 'DT'),
 ('room', 'NN'),
 ('was', 'VBD'),
 ('spacious', 'JJ'),
 ('and', 'CC'),
 ('spacious', 'JJ'),
 (',', ','),
 ('and', 'CC'),
 ('the', 'DT'),
 ('two', 'CD'),
 ('bedrooms', 'NNS'),
 ('could', 'MD'),
 ('be', 'VB'),
 ('seen', 'VBN'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('window.When', 'NN'),
 ('we', 'PRP'),
 ('went', 'VBD'),
 ('there', 'RB'),
 (',', ','),
 ('we', 'PRP'),
 ('caught', 'VBD'),
 ('the', 'DT'),
 ('typhoon', 'NN'),
 ('and', 'CC'),
 ('fortunately', 'RB'),
 ('the', 'DT'),
 ('room', 'NN'),
 ('was', 'VBD'),
 ('occupied', 'VBN'),
 ('with', 'IN'),
 ('doubledecked', 'JJ'),
 ('windows.The', 'NN'),
 ('room', 'NN'),
 ('is', 'VBZ'),
 ('modern', 'JJ'),
 ('and', 'CC'),
 ('technological.Overall', 'NN'),
 (',', ','),
 ('its', 'PRP$'),
 ('pretty', 'RB'),
 ('good', 'JJ'),
 ('.', '.')]

In [67]:
for home_id,rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        tmp_lst = []
        for sent_bow in rev[1]:
            sent_pos = nltk.pos_tag(sent_bow)
            tmp_lst.append(sent_pos)
        rev[1] = tmp_lst

100%|████████████████████████████████████████████████████████████████████████████| 32957/32957 [24:44<00:00, 22.21it/s]


### 데이터 구조를 보자

- review_en == {home_id: [rev0, rev1, ...]}
    - rev0 == [rating, [sent0, sent1, ...]]
        - sent0 == [(word0,POS0),(word1,POS1), ...]
    

In [105]:
k_to_drop = []
for k,rev_lst in review_en.items():
    if review_en[k] == []:
        k_to_drop.append(k)

In [106]:
for k in k_to_drop:
    del review_en[k]

In [107]:
len(review_en) # 리뷰없는 방 제거 후 

22886

In [108]:
# with open('review_en_pos_0113.dic','wb') as fp:
#     pickle.dump(review_en,fp)

In [68]:
review_en['10022410'][0] # rev0의 예시

[5,
 [[('Very', 'RB'), ('good', 'JJ'), ('.', '.')],
  [('The', 'DT'),
   ('landlord', 'NN'),
   ('is', 'VBZ'),
   ('very', 'RB'),
   ('enthusiastic', 'JJ'),
   ('.', '.')],
  [('Its', 'PRP$'),
   ('a', 'DT'),
   ('little', 'JJ'),
   ('bug', 'NN'),
   ('in', 'IN'),
   ('the', 'DT'),
   ('house', 'NN'),
   ('.', '.')]]]

unpacking과 zip 함수 이용해 rating과 review를 각각을 뽑아 리스트로 만들 수도 있다

In [80]:
rating_lst, rev_lst = zip(*review_en['10022410'])

In [82]:
rating_lst

(5, 5, 5, 5, 3, 5, 2, 5, 5, 2, 5, 5, 5, 4, 5, 5, 3, 4, 5, 5, 4, 4, 5)

In [71]:
rev_lst[0]

[[('Very', 'RB'), ('good', 'JJ'), ('.', '.')],
 [('The', 'DT'),
  ('landlord', 'NN'),
  ('is', 'VBZ'),
  ('very', 'RB'),
  ('enthusiastic', 'JJ'),
  ('.', '.')],
 [('Its', 'PRP$'),
  ('a', 'DT'),
  ('little', 'JJ'),
  ('bug', 'NN'),
  ('in', 'IN'),
  ('the', 'DT'),
  ('house', 'NN'),
  ('.', '.')]]

In [109]:
ratings = []
review_wo_ratings = []
for k,rev_lst in review_en.items():
    tmp_rating, tmp_rev = zip(*review_en[k])
    ratings += tmp_rating
    review_wo_ratings += tmp_rev

In [110]:
len(ratings)

383823

In [111]:
len(review_wo_ratings)

383823

In [115]:
# with open('ratings_0113.lst','wb') as fp:
#     pickle.dump(ratings,fp)

In [116]:
# with open('review_wo_ratings_0113.lst','wb') as fp:
#     pickle.dump(review_wo_ratings,fp)

In [18]:
# with open('pos_lst_review.json','w') as fp:
#     json.dump(review_en,fp)

In [117]:
with open('review_en_pos_0113.dic','rb') as fp:
    review_en = pickle.load(fp)

In [118]:
list(review_en.items())[1]

('10076276',
 [[5,
   [[('It', 'PRP'),
     ('was', 'VBD'),
     ('so', 'RB'),
     ('comfortable', 'JJ'),
     ('and', 'CC'),
     ('nice.Haha', 'JJ')]]],
  [3,
   [[('In', 'IN'),
     ('addition', 'NN'),
     ('to', 'TO'),
     ('a', 'DT'),
     ('month', 'NN'),
     (',', ','),
     ('I', 'PRP'),
     ('stayed', 'VBD'),
     ('there', 'EX'),
     ('with', 'IN'),
     ('my', 'PRP$'),
     ('wife', 'NN'),
     ('who', 'WP'),
     ('joined', 'VBD'),
     ('us', 'PRP'),
     ('in', 'IN'),
     ('the', 'DT'),
     ('middle', 'NN'),
     ('of', 'IN'),
     ('2', 'CD'),
     ('weeks.The', 'JJ'),
     ('location', 'NN'),
     ('is', 'VBZ'),
     ('12', 'CD'),
     ('minutes', 'NNS'),
     ('walk', 'VBP'),
     ('from', 'IN'),
     ('Nagaaki', 'NNP'),
     ('station.The', 'NN'),
     ('building', 'NN'),
     ('is', 'VBZ'),
     ('located', 'VBN'),
     ('on', 'IN'),
     ('the', 'DT'),
     ('17th', 'JJ'),
     ('floor', 'NN'),
     ('with', 'IN'),
     ('a', 'DT'),
     ('large', 'JJ'),
   

고유명사 제외한다

In [119]:
NNP_lst = []
for k,rev_lst in review_en.items():
    for rev in rev_lst:
        for sent in rev[1]:
            for w in sent:
                if w[1] == 'NNP' or w[1] == 'NNPS':
                    NNP_lst.append(w)

In [120]:
len(NNP_lst)

457992

In [129]:
sorted(Counter(NNP_lst).items(),key=lambda kv:kv[1],reverse=True)[:30]

[(('Thank', 'NNP'), 29523),
 (('Jeju', 'NNP'), 20771),
 (('Im', 'NNP'), 20529),
 (('Station', 'NNP'), 13274),
 (('Ive', 'NNP'), 11047),
 (('Seoul', 'NNP'), 10793),
 (('Busan', 'NNP'), 10548),
 (('Island', 'NNP'), 9227),
 (('Ill', 'NNP'), 9143),
 (('Id', 'NNP'), 6572),
 (('First', 'NNP'), 6547),
 (('Haeundae', 'NNP'), 6479),
 (('Airbnb', 'NNP'), 5910),
 (('Hanok', 'NNP'), 4167),
 (('Village', 'NNP'), 4039),
 (('Korea', 'NNP'), 3801),
 (('Beach', 'NNP'), 3627),
 (('Daegu', 'NNP'), 2953),
 (('Please', 'NNP'), 2857),
 (('Lotte', 'NNP'), 2785),
 (('Gangnam', 'NNP'), 2531),
 (('Jeonju', 'NNP'), 2521),
 (('University', 'NNP'), 2408),
 (('Hongdae', 'NNP'), 2365),
 (('Gangneung', 'NNP'), 2142),
 (('Seongsan', 'NNP'), 2079),
 (('Yeosu', 'NNP'), 2070),
 (('Nice', 'NNP'), 1959),
 (('Gwangalli', 'NNP'), 1875),
 (('Itaewon', 'NNP'), 1848)]

In [ ]:
#NNP 제거??

TF-IDF 적용해 중요한 단어만 남긴다

In [56]:
# words = []
# for w, rev_lst in review_en.items():
#     for rev in rev_lst:
#         for sent in rev[1]:
#             for w in sent:
#                 if w[1] == 'NNP' or w[1] == 'NNPS':
#                     continue
#                 else:
#                     words.append(w)

In [59]:
# ws,tags = zip(*words)

In [60]:
# unq_words = set(ws)

In [61]:
# len(unq_words)

175632

In [62]:
doc_tokens = {}
maxCount = {}

for i,lst in enumerate(unq_words):
    maxCount[i] = 0
    tmp_tokens = {}
    
    for w in lst:
        if w in tmp_tokens.keys():
            tmp_tokens[w] += 1
        else:
            tmp_tokens[w] = 1

        if maxCount[i] < tmp_tokens[w]:
            maxCount[i] = tmp_tokens[w]
                    
    doc_tokens[i] = tmp_tokens


명사, 형용사, 동사, 부사를 뽑아 lemmantization 시켜주자

함수 njvr_lemmantizer를 정의한다


In [67]:
lemm = WordNetLemmatizer()
def njvr_lemmantizer(sent):
    global lemm
    
    lemm_sent = []
    for word_pos in sent:
        word,pos=word_pos
        if pos[0] == 'N':
            lemm_sent.append(lemm.lemmatize(word,pos='n')+'_N')
        elif pos[0] == 'J':
            lemm_sent.append(lemm.lemmatize(word,pos='a')+'_J')
        elif pos[0] == 'V':
            lemm_sent.append(lemm.lemmatize(word,pos='v')+'_V')
        elif pos[0] == 'R':
            lemm_sent.append(lemm.lemmatize(word,pos='r')+'_R')
        else:pass
    return lemm_sent

샘플 sentence

In [63]:
review_en['10022410'][1][1][0]

[('We', 'PRP'),
 ('had', 'VBD'),
 ('a', 'DT'),
 ('wonderful', 'JJ'),
 ('stay', 'NN'),
 ('at', 'IN'),
 ('Kyungsoon', 'NNP'),
 ('’', 'NNP'),
 ('s', 'JJ'),
 ('place', 'NN'),
 (',', ','),
 ('although', 'IN'),
 ('he', 'PRP'),
 ('can', 'MD'),
 ('only', 'RB'),
 ('speak', 'VB'),
 ('a', 'DT'),
 ('little', 'JJ'),
 ('of', 'IN'),
 ('English', 'JJ'),
 ('but', 'CC'),
 ('he', 'PRP'),
 ("'s", 'VBZ'),
 ('always', 'RB'),
 ('trying', 'VBG'),
 ('to', 'TO'),
 ('help', 'VB'),
 ('whatever', 'VB'),
 ('he', 'PRP'),
 ('can', 'MD'),
 ('.', '.')]

lemmantization 적용후

In [68]:
njvr_lemmantizer(review_en['10022410'][1][1][0])

['have_V',
 'wonderful_J',
 'stay_N',
 'Kyungsoon_N',
 '’_N',
 's_J',
 'place_N',
 'only_R',
 'speak_V',
 'little_J',
 'English_J',
 "'s_V",
 'always_R',
 'try_V',
 'help_V',
 'whatever_V']

전체 리뷰에 적용하자

In [88]:
for listing_id, rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        rev_tmp = []
        for sent in rev[1]:
            rev_tmp.append(njvr_lemmantizer(sent))
        rev[1] = rev_tmp

In [91]:
list(review_en.items())[1][1][1]

[5,
 [['be_V', 'great_J', 'experience_N', 'stay_V', 'Jin_N', 'home_N'],
  ['Jin_N', 'mother_N', 'brother_N', 'be_V', 'so_R', 'good_J', 'friendly_J'],
  ['be_V', 'feel_V', 'be_V', 'stay_V', 'family_N'],
  ['Jin_N',
   'mother_N',
   'have_V',
   'wide_J',
   'variety_N',
   'cook_V',
   'utensil_N',
   'beautiful_J',
   'kitchen_N'],
  ['Sometimes_R', 'offer_V', 'tasty_J', 'Korean_J', 'food_N'],
  ['Location_N',
   'home_N',
   'be_V',
   'situate_V',
   'minute_N',
   'walk_V',
   'distance_N',
   'Pusan_N',
   'national_J',
   'university_N'],
  ['be_V', 'so_R', 'many_J', 'mart_N', 'nearby_R', 'apartment_N'],
  ['Last_J', 'not_R', 'least_J', 'miss_V', 'Jin_N', 'family_N', 'home_N']]]

In [92]:
with open('review_en.json','w') as fp:
    json.dump(review_en,fp)

데이터수집 완료 후 전처리

In [4]:
with open('pos_review.json') as fp:
    review = json.load(fp)

In [5]:
len(review)

30569

In [7]:
voca_freq_dic = {}
for rev_lst in review.values():
    for rev in rev_lst:
        for sent in rev[1]:
            for word in sent:
                if word in voca_freq_dic:
                    voca_freq_dic[word] += 1
                else:
                    voca_freq_dic[word] = 1

In [8]:
len(voca_freq_dic)

147119

In [10]:
sorted(voca_freq_dic.items(),key=lambda kv:kv[1],reverse=True)[:10]

[('be_V', 1652787),
 ('very_R', 346535),
 ('have_V', 231030),
 ('place_N', 229608),
 ('good_J', 228951),
 ('clean_J', 181324),
 ('so_R', 163087),
 ('nice_J', 155968),
 ('room_N', 155617),
 ('host_N', 146170)]

In [12]:
with open('voca_freq_dic.json','w') as fp:
    json.dump(voca_freq_dic,fp)